<a href="https://colab.research.google.com/github/lihiShlomit/DSproject/blob/main/trainYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17516 (delta 6), reused 0 (delta 0), pack-reused 17497 (from 4)
Receiving objects: 100% (17516/17516), 16.62 MiB | 20.71 MiB/s, done.
Resolving deltas: 100% (12001/12001), done.
/content/yolov5/yolov5/yolov5/yolov5


In [7]:
!ls utils

activations.py	  callbacks.py	  general.py	     metrics.py
augmentations.py  dataloaders.py  google_app_engine  plots.py
autoanchor.py	  docker	  __init__.py	     segment
autobatch.py	  downloads.py	  loggers	     torch_utils.py
aws		  flask_rest_api  loss.py	     triton.py


In [8]:
import sys
sys.path.append('/content/yolov5')

import torch
from models.yolo import Model
from utils.dataloaders import create_dataloader
from utils.general import check_dataset
from utils.torch_utils import select_device
from utils.loss import ComputeLoss
from pathlib import Path

In [9]:
from google.colab import drive
drive.mount('/content/drive')

DATA_YAML_PATH = "/content/drive/MyDrive/project/data/license_plates/archive/dataset.yaml"

check_dataset(DATA_YAML_PATH)

Mounted at /content/drive


100%|██████████| 755k/755k [00:00<00:00, 14.7MB/s]


{'train': '/content/drive/MyDrive/project/data/license_plates/archive/images/train',
 'val': '/content/drive/MyDrive/project/data/license_plates/archive/images/val',
 'nc': 1,
 'names': {0: 'license_plate'},
 'path': PosixPath('/content/yolov5')}

In [10]:

hyp = {
    'lr0': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 0.05,
    'cls': 0.5,
    'obj': 1.0,
    'anchor_t': 4.0,
    'fl_gamma': 0.0,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.0,
    'copy_paste': 0.0,
    'cls_pw': 1.0,
    'obj_pw': 1.0
}

cfg = 'models/yolov5s.yaml'
nc = 1
device = select_device('')
model = Model(cfg, ch=3, nc=nc).to(device)

imgsz = 640
train_path = "/content/drive/MyDrive/project/data/license_plates/archive/images/train"
train_loader, dataset = create_dataloader(
    path=train_path,
    imgsz=imgsz,
    batch_size=16,
    stride=32,
    hyp=hyp,
    augment=True,
    rect=False,
    cache=False,
    single_cls=False,
    pad=0.5,
    prefix='Train: '
)

optimizer = torch.optim.SGD(model.parameters(), lr=hyp['lr0'],
                            momentum=hyp['momentum'], weight_decay=hyp['weight_decay'])
model.hyp = hyp
compute_loss = ComputeLoss(model)


epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, (imgs, targets, paths, _) in enumerate(train_loader):
      if i>20:
        break
      imgs = imgs.to(device).float() / 255.0
      targets = targets.to(device)
      pred = model(imgs)
      loss, loss_items = compute_loss(pred, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CPU

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  mod

Epoch 1/10 - Loss: 0.4673
Epoch 2/10 - Loss: 0.4120
Epoch 3/10 - Loss: 0.3779
Epoch 4/10 - Loss: 0.3570
Epoch 5/10 - Loss: 0.3231
Epoch 6/10 - Loss: 0.3239
Epoch 7/10 - Loss: 0.3141
Epoch 8/10 - Loss: 0.3100
Epoch 9/10 - Loss: 0.2971
Epoch 10/10 - Loss: 0.2929
Model saved.
